In [ ]:
import os
import matplotlib.pyplot as plt
current_dir = os.getcwd()
new_dir = current_dir.replace("\\tests", "")
os.chdir(new_dir)
from services.medias.video_processor import VideoProcessor
from DTOs.person_dto import PersonDTO
from models.medias.video import Video

In [ ]:
video: Video = Video("training/datasets/detection/test/videos/QUI EST L IMPOSTEUR (ft Eric Ramzy).mp4")
video_processor: VideoProcessor = VideoProcessor(video)
personsDTO: list[PersonDTO] = video_processor.get_persons()
for personDTO in personsDTO:
    plt.suptitle("Persons detected")
    plt.subplot(1, len(personsDTO), personDTO.id+1)
    plt.imshow(personDTO.face)
    plt.axis('off')

In [ ]:
# Blur

for personDTO in personsDTO:
    personDTO.should_be_blurred = True
video_processor.save(personsDTO=personsDTO, output_video_path="results/output.mp4", gradual=True)

In [3]:
import tensorflow as tf

In [12]:
if tf.test.gpu_device_name():
    print("TensorFlow is using GPU:", tf.test.gpu_device_name())
else:
    print("TensorFlow is using CPU")

TensorFlow is using GPU: /device:GPU:0
